In [ ]:
import numpy as np

# (1) 상태와 행동 정의
states = ["S", "R1", "R2", "R3", "F"]
actions = ["A1", "A2"]
gamma = 0.5  # 감가율

# (2) 정책 π(s, a): 상태에서 행동을 선택할 확률
policy = {
    "S": {"A1": 0.6, "A2": 0.4},
    "R1": {"A1": 0.7, "A2": 0.3},
    "R2": {"A1": 1.0},
    "R3": {"A1": 1.0}
}

# (3) 상태 전이 확률 P(s, a, s')
transition_probs = {
    ("S", "A1"): "R1",
    ("S", "A2"): "R2",
    ("R1", "A1"): "R3",
    ("R1", "A2"): "R2",
    ("R2", "A1"): "R3",
    ("R3", "A1"): "F"
}

# (4) 보상 함수 R(s, a)
rewards = {
    ("S", "A1"): 0.5,
    ("S", "A2"): 1.5,
    ("R1", "A1"): 1.0,
    ("R1", "A2"): 1.5,
    ("R2", "A1"): 2.0,
    ("R3", "A1"): 3.0
}

# (5) 에피소드 시뮬레이션 함수(정책 기반 경로)
def simulate_episode(start_state="S"):
    state = start_state
    total_return = 0
    discount = 1.0

    while state != "F":
        action_probs = policy[state]
        actions_list = list(action_probs.keys())
        probs = list(action_probs.values())
        action = np.random.choice(actions_list, p=probs)

        reward = rewards.get((state, action), 0)
        total_return += discount * reward

        next_state = transition_probs.get((state, action), "F")
        state = next_state
        discount *= gamma

    return total_return

n_episodes = 10000
state_values = {}
q_values = {}

# (6) 상태가치함수 계산
for state in states:
    episode_returns = []
    for _ in range(n_episodes):
        result = simulate_episode(start_state=state)
        episode_returns.append(result)
    state_values[state] = np.mean(episode_returns)

# (7) 행동가치함수 계산
for state in policy:
    for action in policy[state]:
        returns = []
        for _ in range(n_episodes):
            temp_state = state
            total_return = 0
            discount = 1.0

            reward = rewards.get((temp_state, action), 0)
            total_return += discount * reward
            next_state = transition_probs.get((temp_state, action), "F")
            temp_state = next_state
            discount *= gamma

            while temp_state != "F":
                action_probs = policy[temp_state]
                actions_list = list(action_probs.keys())
                probs = list(action_probs.values())
                next_action = np.random.choice(actions_list, p=probs)

                reward = rewards.get((temp_state, next_action), 0)
                total_return += discount * reward
                temp_state = transition_probs.get((temp_state, next_action), "F")
                discount *= gamma

            returns.append(total_return)
        q_values[(state, action)] = np.mean(returns)

# (8) 결과 출력
print("\n✅ 상태가치함수 Vπ(s):")
for s, v in state_values.items():
    print(f"  {s}: {v:.4f}")

print("\n✅ 행동가치함수 Qπ(s,a):")
for (s, a), q in q_values.items():
    print(f"  ({s}, {a}): {q:.4f}")

# (9) 최적정책 계산(가장 Q값이 높은 행동만 선택)
optimal_policy = {}
for state in policy:
    best_action = None
    best_q_value = float("-inf")

    # 가능한 모든 행동에 대해 Q값을 비교해서 가장 큰 것 선택
    for action in policy[state]:
        q = q_values.get((state, action), float("-inf"))
        if q > best_q_value:
            best_q_value = q
            best_action = action

    # 최적정책: 가장 좋은 행동만 확률 1로 설정
    action_probabilities = {}
    for action in policy[state]:
        if action == best_action:
            action_probabilities[action] = 1.0
        else:
            action_probabilities[action] = 0.0

    optimal_policy[state] = action_probabilities

# (10) 최적정책 출력
print("\n🌟 최적정책 π*(s):")
for state in optimal_policy:
    for action, prob in optimal_policy[state].items():
        if prob == 1.0:
            print(f"  상태 {state} → 최적 행동: {action}")


✅ 상태가치함수 Vπ(s):
  S: 2.4114
  R1: 2.7238
  R2: 3.5000
  R3: 3.0000
  F: 0.0000

✅ 행동가치함수 Qπ(s,a):
  (S, A1): 1.8616
  (S, A2): 3.2500
  (R1, A1): 2.5000
  (R1, A2): 3.2500
  (R2, A1): 3.5000
  (R3, A1): 3.0000

🌟 최적정책 π*(s):
  상태 S → 최적 행동: A2
  상태 R1 → 최적 행동: A2
  상태 R2 → 최적 행동: A1
  상태 R3 → 최적 행동: A1
